## Deliverable 2. Create a Customer Travel Destinations Map.

In [4]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Cape Town,ZA,-33.9258,18.4232,69.35,63,34,3.42,scattered clouds,2022-01-16 21:55:07
1,1,Ponta Do Sol,PT,32.6667,-17.1000,65.03,62,84,18.48,broken clouds,2022-01-16 21:51:53
2,2,Norman Wells,CA,65.2820,-126.8329,-25.56,68,20,0.00,few clouds,2022-01-16 21:56:32
3,3,Bubaque,GW,11.2833,-15.8333,78.55,82,99,12.37,overcast clouds,2022-01-16 21:49:39
4,4,Vaini,TO,-21.2000,-175.2000,80.11,82,30,12.88,scattered clouds,2022-01-16 21:56:32


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
3,3,Bubaque,GW,11.2833,-15.8333,78.55,82,99,12.37,overcast clouds,2022-01-16 21:49:39
4,4,Vaini,TO,-21.2000,-175.2000,80.11,82,30,12.88,scattered clouds,2022-01-16 21:56:32
7,7,Butaritari,KI,3.0707,172.7902,82.38,77,35,16.64,scattered clouds,2022-01-16 21:56:33
11,11,Kroya,ID,-7.6331,109.2461,76.87,88,41,3.06,scattered clouds,2022-01-16 21:56:34
12,12,Puerto Ayora,EC,-0.7393,-90.3518,84.16,78,100,10.60,overcast clouds,2022-01-16 21:56:35
13,13,Canberra,AU,-35.2835,149.1281,76.44,80,68,5.75,broken clouds,2022-01-16 21:55:12
16,16,Rikitea,PF,-23.1203,-134.9692,77.25,69,35,7.49,scattered clouds,2022-01-16 21:56:35
17,17,Chuy,UY,-33.6971,-53.4616,77.27,82,99,5.79,overcast clouds,2022-01-16 21:56:36
19,19,Hilo,US,19.7297,-155.0900,80.11,78,1,5.99,clear sky,2022-01-16 21:56:06
21,21,Flinders,AU,-34.5833,150.8552,76.37,86,16,4.00,few clouds,2022-01-16 21:48:23


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()


City_ID                172
City                   172
Country                172
Lat                    172
Lng                    172
Max Temp               172
Humidity               172
Cloudiness             172
Wind Speed             172
Current Description    172
Date                   172
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.count()

City_ID                172
City                   172
Country                172
Lat                    172
Lng                    172
Max Temp               172
Humidity               172
Cloudiness             172
Wind Speed             172
Current Description    172
Date                   172
dtype: int64

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Bubaque,GW,78.55,overcast clouds,11.2833,-15.8333,
4,Vaini,TO,80.11,scattered clouds,-21.2000,-175.2000,
7,Butaritari,KI,82.38,scattered clouds,3.0707,172.7902,
11,Kroya,ID,76.87,scattered clouds,-7.6331,109.2461,
12,Puerto Ayora,EC,84.16,overcast clouds,-0.7393,-90.3518,
13,Canberra,AU,76.44,broken clouds,-35.2835,149.1281,
16,Rikitea,PF,77.25,scattered clouds,-23.1203,-134.9692,
17,Chuy,UY,77.27,overcast clouds,-33.6971,-53.4616,
19,Hilo,US,80.11,clear sky,19.7297,-155.0900,
21,Flinders,AU,76.37,few clouds,-34.5833,150.8552,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [13]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()
hotel_df.count()

City                   172
Country                172
Max Temp               172
Current Description    172
Lat                    172
Lng                    172
Hotel Name             172
dtype: int64

In [20]:
# 8a. Create the output File (CSV)
# Create the output file (CSV).
output_data_file = "../Weather_Database/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [24]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))